In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from azure.storage.blob import BlobService
import pyspark
import pandas
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import time
import pandas
import datetime, time
from pyspark.sql import functions as F
from math import sqrt
from operator import add
conf = (SparkConf().set("spark.driver.maxResultSize", "64G"))
print conf.get("spark.num-executors")
conf = (conf.set("spark.num-executors", "20"))
conf = (conf.set("spark.driver-cores", "7"))
conf = (conf.set("spark.num-executors", "40"))
conf = (conf.set("spark.executor-cores", "4"))
conf = (conf.set("spark.executor-memory", "64G"))
conf = (conf.set("spark.driver-memory", "64G"))
spark = SparkSession.builder.master("local").appName("do_mf").config(conf=conf).getOrCreate()
print conf.get("spark.driver.maxResultSize")
sc.setLogLevel('ERROR')

Read Churn data

In [ ]:
block_blob_service_cltv = BlobService(account_name='asoscustprofdata', account_key='gWadTN6sXXsLIurQxCjJSFo/Ab6MNsAmpyauMImlOuIe5JI928yP9d3bNhL64IdSmL9spqPHz658iFcr2J56zA==')
df = spark.read.parquet("wasbs://fullchurnltvsampledataset@asoscustprofdata.blob.core.windows.net/2017-02-24/historical/")

In [ ]:
df = df.select(df["customerId"], df["shippingCountry"], df["gender"], df["target_net_value"], df["target_churned"])


Read Viewings data

In [ ]:
block_blob_service_views = BlobService(account_name='asoscustprofdevelopment', account_key='X1nmVm/GGg+UQLrDHBTfqxNRFvD4XI1GjnMCF2C2DvWT8pu+3qVI3BVt+2t2k0Ksz1PNPRoHgpqC9QOxmmZJcg==')

In [ ]:
viewsSchema = StructType([StructField("customerId", IntegerType(), False),
                        StructField("productId", IntegerType(), True),
                        StructField("variantId", IntegerType(), True),
                        StructField("divisionId", IntegerType(), True),
                        StructField("sourceId", IntegerType(), True),
                        StructField("itemQty", IntegerType(), True), # always 1
                        StructField("signalDate", TimestampType(), True), # time of viewing
                        StructField("sessionId", StringType(), True), # session id
                        StructField("price", DoubleType(), True),
                        StructField("useForRecs", IntegerType(), True),
                        StructField("discountType", IntegerType(), True),
                        StructField("dateModified", TimestampType(), True),
                        StructField("viewonly", IntegerType(), True), # nr of times user has clicked on product
                        StructField("changeThumbnail", IntegerType(), True), # count nr of img thumbnail clicks
                        StructField("imageZoom", IntegerType(), True),
                        StructField("watchVideo", IntegerType(), True),
                        StructField("view360", IntegerType(), True),
                        StructField("sizeGuide", IntegerType(), True),
                        StructField("device", StringType(), True),
                        StructField("userTime", TimestampType(), True) # time in user time zone
                         ])
apps1 = spark.read.csv("wasbs://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/sample1outof10singlefile/app_productviews", schema=viewsSchema, sep="\t")
#apps1.count()

In [ ]:
desktop1 = spark.read.csv("wasbs://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/sample1outof10singlefile/desktop_productviews", schema=viewsSchema, sep="\t")
#desktop1.count()

In [ ]:
apps1 = apps1.select("customerId", "productId", "signalDate", "divisionId")
desktop1 = desktop1.select("customerId", "productId", "signalDate", "divisionId")

In [ ]:
apps1 = apps1.filter(apps1.signalDate >= datetime.datetime(2015, 2, 24, 9, 0, 0))
apps1 = apps1.filter(apps1.signalDate < datetime.datetime(2016, 2, 24, 9, 0, 0))

desktop1 = desktop1.filter(desktop1.signalDate >= datetime.datetime(2015, 2, 24, 9, 0, 0))
desktop1 = desktop1.filter(desktop1.signalDate < datetime.datetime(2016, 2, 24, 9, 0, 0))

In [ ]:
views_all = apps1.union(desktop1)

In [ ]:
views = df.join(views_all, df.customerId == views_all.customerId, 'inner').drop(views_all.customerId)

In [ ]:
views = views.filter(views.shippingCountry == "UK")

Count unique customers and products

In [ ]:
un = views.select(views["customerId"]).distinct()
un.count()

In [ ]:
un_p = views.select(views["productId"]).distinct()
un_p.count() 

In [ ]:
df.createOrReplaceTempView("df")
views.createOrReplaceTempView("views")

In [ ]:
views_df_ordered = spark.sql("SELECT customerId, productId, signalDate FROM views ORDER BY productId, signalDate")


Write data

In [ ]:
labels_write = spark.sql("SELECT DISTINCT(customerId), gender, target_net_value, target_churned FROM views")

In [ ]:
labels_write.coalesce(1).write.csv("wasb://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/customer_labels", sep="\t")

In [ ]:
views_df_ordered.coalesce(1).write.csv("wasb://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/customer_views", sep="\t")